In [0]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"



Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 321 (delta 4), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (321/321), 1.47 MiB | 11.25 MiB/s, done.
Resolving deltas: 100% (187/187), done.


# Paqueterias y funciones 



Instalamos las paqueterias que vamos a utilizar


In [0]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12-0.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Content type 'application/x-gzip'
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 540698 bytes (528 KB)

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: =
  warnings.w

Describimos las funciones que vamos a utilizar


In [0]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}
#otras funciones que utilizaremos

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
}




# Definición de parámetros para valorar:








Cargamos parámetros para valoración: 
Fecha de valuacion 28 de febrero 2020
Nivel de confianza: 0.98
Y con los siguientes contratos

In [0]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia


#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_eq=c(1000,5000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_fx=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

#CETES
base="RiesgosFinancieros/2020-2/Tarea/tasa_guber.txt"

#BONDES D
btasadescst="RiesgosFinancieros/2020-2/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-2/Tarea/tfondeo.txt"
plazos_bdm=cbind( 180,	3600,	707) #Vencimiento del bono
plazocupon_bdm=cbind( 28,	28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(1000, 1500, 1500) #posición invertida
nominal_bdm=100

#FORWARDS TDC (como es tipo de cambio en el cual intervienen las divisas, lo llamé como pos_fx) 
bext="RiesgosFinancieros/2020-2/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2020-2/Tarea/tasa_fwd.txt"
btsp="RiesgosFinancieros/2020-2/Tarea/tasa_spot.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind(5)
contratos_fwd=cbind(100)
pos_fx <- contratos_fwd
kst_fwd=cbind(19.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
#base="RiesgosFinancieros/2020-2/Tarea/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53)#plazo
contratos_fwd_ind=cbind(50)#numero de
kst_fwd_ind=cbind(-49525)
nominal_fwd_ind=1

#SWAP
btasadesc_sw="RiesgosFinancieros/2020-2/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2020-2/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.052,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(1,1) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1600,1200) #se establece el nominal_sw de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

#OPCIONES
btasadesc_oir="RiesgosFinancieros/2020-2/Tarea/tasa_TIIE_SW_OP.txt"
btasaspot_oir="RiesgosFinancieros/2020-2/Tarea/tasa_DIRS_SW_OP.txt"
bvolspot_oir="RiesgosFinancieros/2020-2/Tarea/tvoltiie_opc.txt"
plazos_oir=cbind(1700,700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.094, 0.089)
contratos_oir=cbind(1000,500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple


# Carga de datos

In [0]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_fx),t(pos_eq))
start_date=Sys.Date()-nh #fecha inicial
#Creación del objeto para guardar los datos
dataEnv<-new.env()
dataEnvFX<-new.env()
#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)
getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date)
#muestra
#tail(dataEnvFX$`GBPUSD=X`)
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(dataEnv,align='remove.na',fill.gaps = T)
bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
#muestra de datos
#head(dataEnv$prices)
#head(dataEnvFX$prices)
#Nos quedamos con los precios
stock_prices = dataEnv$prices
#tail(stock_prices[,])
stock_pricesFX=dataEnvFX$prices
#tail(stock_pricesFX)
#cambiar todo a pesos mexicanos
stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
#tail(stock_pricesFX)
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")
#stock_prices_EQFX
#tail(stock_prices_EQFX)
#Preciso actuales
#x0=as.data.table(as.matrix(stock_prices_EQFX[nrow(stock_prices_EQFX),])) #valores actuales
x0=stock_prices_EQFX[nrow(stock_prices_EQFX),]
#x0

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))
#LEER DATOS DE CETES #CARGA DE DATOS DE BONO CUPÓN CERO
data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

##CARGA DE DATOS DE BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
print(head(X2_orig_bd))
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
print(head(X2_orig_bd))
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.

print(head(X1_orig[,1:4]))


##############################################

#CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  
  
  #CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])


          V1         V2         V3         V4
1 2020-03-06 0.07906000 0.07808887 0.07709264
2 2020-03-05 0.07722139 0.07808887 0.07709264
3 2020-03-04 0.07722139 0.07808887 0.07744709
4 2020-03-03 0.07813334 0.07808887 0.07744709
5 2020-03-02 0.07722139 0.07808887 0.07744709
6 2020-02-28 0.07722139 0.07802004 0.07744709
        V1    V2
2 19981103 30.75
3 19981104  29.2
4 19981105  29.8
5 19981106  31.3
6 19981109  32.9
7 19981110 34.54
           V1   V2       Date   ID      fecha
1: 2020-03-13 7.06 2020-03-13 7802 2020-03-13
2: 2020-03-12 7.10 2020-03-12 7801 2020-03-12
3: 2020-03-11 7.02 2020-03-11 7800 2020-03-11
4: 2020-03-10 6.98 2020-03-10 7799 2020-03-10
5: 2020-03-09 7.05 2020-03-09 7798 2020-03-09
6: 2020-03-06 7.04 2020-03-08 7797 2020-03-08
           V1         V2         V3         V4
1: 2020-03-06 0.07906000 0.07808887 0.07709264
2: 2020-03-05 0.07722139 0.07808887 0.07709264
3: 2020-03-04 0.07722139 0.07808887 0.07744709
4: 2020-03-03 0.07813334 0.07808887 0.07744709
5:

# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [0]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
#head(x_orig_gov)  
lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub_bmybdst=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
#lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol
print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)


           Date
  1: 2020-03-06
  2: 2020-03-05
  3: 2020-03-04
  4: 2020-03-03
  5: 2020-03-02
 ---           
309: 2019-03-26
310: 2019-03-25
311: 2019-03-21
312: 2019-03-20
313: 2019-03-19


In [0]:
%%R
#n=nrow(lin_gub_bmybdst_flibfwdspind) #Historia de todos
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)

#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)


#historia de swaps
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

# Pasos para medir el riesgo por el método de Simulación Histórica

Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Valoración al día actual

##Acciones y divisas


In [0]:

%%R
#divisas y acciones CÁLCULO

x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
m_fx=length(pos_fx)
m_acc=length(pos_eq)
print(V0_acc_div)

   EURUSD.X GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1:  2223.95 25661.13 99072.51   19104        5294     57500


##Bondes D

In [0]:
%%R
#BONDE D CÁLCULO

X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 



V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
print(V0_bd)

         [,1]     [,2]     [,3]
[1,] 98774.93 129547.7 142895.4


##Swaps

In [0]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)

print(V0_sw)


        [,1]      [,2]
[1,] 64.3199 -28.35638


##Opciones de tasas de interes

In [0]:
%%R

##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio

print(V0_oir)


        [,1]      [,2]
[1,] 2.24108 0.1065757


# Integración de factores y cálculo de riesgo en conjunto, y aplicación de simulación

In [0]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,4,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(X_bd_ext) #bonde
n_if[3]=ncol(X_sw) #swaps
n_if[4]=ncol(X_oir) #opciones tasa de interés

#valor del portafolios

V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,X_bd_ext,X_sw,X_oir) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
#alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
#print(head(DeltaX_s))
print(ncol(DeltaX_s))

[1] 562


# Medición de Riesgo 

##Acciones y divisas

In [0]:


%%R
#Medición de riesgo por instrumento, instrumento-factor de riesgo, instrumento - total

#Cálculo de matriz de pérdidas y ganancias Acciones y Divisas

#riesgo del acciones y divisas
m=m_fx+m_acc    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_fx)
Vfr2_acc_div=matrix(0,Ns,m_acc)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_fx)
PGfr2_acc_div=matrix(0,Ns,m_acc)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

DeltaX_s_acc_div=DeltaX_s[,(1:n_if[1])]    #PASO CLAVE
x0_acc_div=stock_prices_EQFX[1,]   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*(1+DeltaX_s_acc_div[i,]))
  #PASO CLAVE
  V_acc_div[i,]=cbind(t(pos_fx),t(pos_eq))*X_s_acc_div[i,]
  #PASO CLAVE
  Vfr1_acc_div[i,]=t(pos_fx)*X_s_acc_div[i,1:m_fx]
  #PASO CLAVE
  Vfr2_acc_div[i,]=t(pos_eq)*X_s_acc_div[i,(m_fx+1):(m_fx+m_acc)]
  #PASO CLAVE
  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)
  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_fx])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_fx+1):(m_fx+m_acc)])
  PGT_acc_div[i,]=sum(PG_acc_div[i,])
  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}



PG_acc_div[1:5,]
PGfr1_acc_div[1:5,]
PGfr2_acc_div[1:5,]
PGT_acc_div[1:5,]


#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_fx)
VaRfr2_acc_div=matrix(0,1,m_acc)
CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_fx)
CVaRfr2_acc_div=matrix(0,1,m_acc)
for (i in (1:m))
{
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  if (i<=m_fx)
  {
  VaRfr1_acc_div[i]=equantile(PGfr1_acc_div[,i],1-alpha,Ns)
  CVaRfr1_acc_div[i]= mean(merge(which(PGfr1_acc_div[,i]<VaRfr1_acc_div[i]),cbind(seq(1,Ns),PGfr1_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
  if (i<=m_acc)
  {  
  VaRfr2_acc_div[i]=equantile(PGfr2_acc_div[,i],1-alpha,Ns)
  CVaRfr2_acc_div[i]= mean(merge(which(PGfr2_acc_div[,i]<VaRfr2_acc_div[i]),cbind(seq(1,Ns),PGfr2_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
}

VaRCont_acc_div
VaRfr1_acc_div
VaRfr2_acc_div
CVaRCont_acc_div
CVaRfr1_acc_div
CVaRfr2_acc_div


#VaR Total

VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])


print(cbind(VaRTotal_acc_div,sum(V0_acc_div), VaRCont_acc_div, V0_acc_div))
cbind(CVaRTotal_acc_div,sum(V0_acc_div), CVaRCont_acc_div, V0_acc_div)
cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div)
cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div)

####hacemos el calculo para el que si tiene alisado

#VaR por posición
VaRCont_CA_acc_div=matrix(0,1,m)
VaRfr1_CA_acc_div=matrix(0,1,m_fx)
VaRfr2_CA_acc_div=matrix(0,1,m_acc)
CVaRCont_CA_acc_div=matrix(0,1,m)
CVaRfr1_CA_acc_div=matrix(0,1,m_fx)
CVaRfr2_CA_acc_div=matrix(0,1,m_acc)
for (i in (1:m))
{
  VaRCont_CA_acc_div[i]=wquantile(PG_acc_div[,i],w=rep(1,length(PG_acc_div[,i])),1-alpha)
  CVaRCont_CA_acc_div[i]= wcvar(PG_acc_div[,i],w=rep(1,length(PG_acc_div[,i])),1-alpha)
  if (i<=m_fx)
  {
  VaRfr1_CA_acc_div[i]=wquantile(PGfr1_acc_div[,i],w=rep(1,length(PGfr1_acc_div[,i])),1-alpha)
  CVaRfr1_CA_acc_div[i]= wcvar(PGfr1_acc_div[,i],w=rep(1,length(PGfr1_acc_div[,i])),1-alpha)
  }
  if (i<=m_acc)
  {  
  VaRfr2_CA_acc_div[i]=wquantile(PGfr2_acc_div[,i],w=rep(1,length(PGfr2_acc_div[,i])),1-alpha)
  CVaRfr2_CA_acc_div[i]= wcvar(PGfr2_acc_div[,i],w=rep(1,length(PGfr2_acc_div[,i])),1-alpha)
  }
}

VaRCont_CA_acc_div
VaRfr1_CA_acc_div
VaRfr2_CA_acc_div
CVaRCont_CA_acc_div
CVaRfr1_CA_acc_div
CVaRfr2_CA_acc_div

####Hasta aqui vamos
#VaR Total

VaRTotal_CA_acc_div=wquantile(PGT_acc_div,w=rep(1,length(PGT_acc_div)),1-alpha)
CVaRTotal_CA_acc_div= wcvar(PGT_acc_div,w=rep(1,length(PGT_acc_div)),1-alpha)
VaRTotalfr1_CA_acc_div=wquantile(PGfr1T_acc_div,w=rep(1,length(PGfr1T_acc_div)),1-alpha)
CVaRTotalfr1_CA_acc_div= wcvar(PGfr1T_acc_div,w=rep(1,length(PGfr1T_acc_div)),1-alpha)
VaRTotalfr2_CA_acc_div=wquantile(PGfr2T_acc_div,w=rep(1,length(PGfr2T_acc_div)),1-alpha)
CVaRTotalfr2_CA_acc_div= wcvar(PGfr2T_acc_div,w=rep(1,length(PGfr2T_acc_div)),1-alpha)


cbind(VaRTotal_CA_acc_div,sum(V0_acc_div), VaRCont_CA_acc_div, V0_acc_div)
cbind(CVaRTotal_CA_acc_div,sum(V0_acc_div), CVaRCont_CA_acc_div, V0_acc_div)
print("VaR con alisado")
print(cbind(VaRTotal_CA_acc_div,VaRTotalfr1_CA_acc_div,VaRTotalfr2_CA_acc_div))
print("CVaR con alisado")
print(cbind(CVaRTotal_CA_acc_div,CVaRTotalfr1_CA_acc_div,CVaRTotalfr2_CA_acc_div))


Error in withVisible({ : 
  dims [product 4] do not match the length of object [6]


/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in withVisible({ : 
  dims [product 4] do not match the length of object [6]

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: In cbind(t(pos_fx), t(pos_eq)) * X_s_acc_div[i, ] :
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  longer object length is not a multiple of shorter object length

  warnings.warn(x, RRuntimeWarning)


##Bondes D

In [0]:
%%R

#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bd=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_bd=matrix(0,Ns,m)
Vfr1_bd=matrix(0,Ns,m)
Vfr2_bd=matrix(0,Ns,m)
Vfr3_bd=matrix(0,Ns,m)
PG_bd=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_bd=matrix(0,Ns,m)
PGfr2_bd=matrix(0,Ns,m)
PGfr3_bd=matrix(0,Ns,m)
PGT_bd=matrix(0,Ns,1)
PGfr1T_bd=matrix(0,Ns,1)
PGfr2T_bd=matrix(0,Ns,1)
PGfr3T_bd=matrix(0,Ns,1)

DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bd=X_bd_ext[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_bd[i,]=x0_bd*(1+DeltaX_s_bd[i,])
  #PASO CLAVE
  V_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr1_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr2_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[4]/3+1):(n_if[4]*2/3)], x0_bd[(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  Vfr3_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[4]/3+1):(n_if[4]*2/3)], X_s_bd[i,(n_if[4]*2/3+1):(n_if[4])], N_bd,ddv)
  #PASO CLAVE
  PG_bd[i,]=V_bd[i,]-V0_bd
  PGfr1_bd[i,]=Vfr1_bd[i,]-V0_bd
  PGfr2_bd[i,]=Vfr2_bd[i,]-V0_bd
  PGfr3_bd[i,]=Vfr3_bd[i,]-V0_bd
  PGT_bd[i,]=sum(PG_bd[i,])
  PGfr1T_bd[i,]=sum(PGfr1_bd[i,])
  PGfr2T_bd[i,]=sum(PGfr2_bd[i,])
  PGfr3T_bd[i,]=sum(PGfr3_bd[i,])
}



PG_bd[1:5,]
PGfr1_bd[1:5,]
PGfr2_bd[1:5,]
PGT_bd[1:5,]


#VaR por posición
VaRCont_bd=matrix(0,1,m)
VaRfr1_bd=matrix(0,1,m)
VaRfr2_bd=matrix(0,1,m)
VaRfr3_bd=matrix(0,1,m)
CVaRCont_bd=matrix(0,1,m)
CVaRfr1_bd=matrix(0,1,m)
CVaRfr2_bd=matrix(0,1,m)
CVaRfr3_bd=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_bd[i]=equantile(PG_bd[,i],1-alpha,Ns)
  VaRfr1_bd[i]=equantile(PGfr1_bd[,i],1-alpha,Ns)
  VaRfr2_bd[i]=equantile(PGfr2_bd[,i],1-alpha,Ns)
  VaRfr3_bd[i]=equantile(PGfr3_bd[,i],1-alpha,Ns)
  CVaRfr1_bd[i]= mean(merge(which(PGfr1_bd[,i]<VaRfr1_bd[i]),cbind(seq(1,Ns),PGfr1_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_bd[i]= mean(merge(which(PGfr2_bd[,i]<VaRfr2_bd[i]),cbind(seq(1,Ns),PGfr2_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_bd[i]= mean(merge(which(PGfr3_bd[,i]<VaRfr3_bd[i]),cbind(seq(1,Ns),PGfr3_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_bd[i]= mean(merge(which(PG_bd[,i]<VaRCont_bd[i]),cbind(seq(1,Ns),PG_bd[,i]), by.x=1,by.y=1)[,2])
}

VaRCont_bd
VaRfr1_bd
VaRfr2_bd
CVaRCont_bd
CVaRfr1_bd
CVaRfr2_bd


#VaR Total

VaRTotal_bd=equantile(PGT_bd,1-alpha,Ns)
CVaRTotal_bd= mean(merge(which(PGT_bd<VaRTotal_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])
VaRTotalfr1_bd=equantile(PGfr1T_bd,1-alpha,Ns)
CVaRTotalfr1_bd= mean(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_bd),])
VaRTotalfr2_bd=equantile(PGfr2T_bd,1-alpha,Ns)
CVaRTotalfr2_bd= mean(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])
VaRTotalfr3_bd=equantile(PGfr3T_bd,1-alpha,Ns)
CVaRTotalfr3_bd= mean(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])


cbind(VaRTotal_bd,sum(V0_bd), VaRCont_bd, V0_bd)
cbind(CVaRTotal_bd,sum(V0_bd), CVaRCont_bd, V0_bd)
print("VaR sin alisado")
print(cbind(VaRTotal_bd,VaRTotalfr1_bd,VaRTotalfr2_bd,VaRTotalfr3_bd))
print("CVaR sin alisado")
print(cbind(CVaRTotal_bd,CVaRTotalfr1_bd,CVaRTotalfr2_bd,CVaRTotalfr3_bd))


###Vamos con el calculo con Alisado

#VaR por posición
VaRCont_CA_bd=matrix(0,1,m)
VaRfr1_CA_bd=matrix(0,1,m)
VaRfr2_CA_bd=matrix(0,1,m)
VaRfr3_CA_bd=matrix(0,1,m)
CVaRCont_CA_bd=matrix(0,1,m)
CVaRfr1_CA_bd=matrix(0,1,m)
CVaRfr2_CA_bd=matrix(0,1,m)
CVaRfr3_CA_bd=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_CA_bd[i]=wquantile(PG_bd[,i],w=rep(1,length(PG_bd[,i])),1-alpha)
  VaRfr1_CA_bd[i]=wquantile(PGfr1_bd[,i],w=rep(1,length(PGfr1_bd[,i])),1-alpha)
  VaRfr2_CA_bd[i]=wquantile(PGfr2_bd[,i],w=rep(1,length(PGfr2_bd[,i])),1-alpha)
  VaRfr3_CA_bd[i]=wquantile(PGfr3_bd[,i],w=rep(1,length(PGfr3_bd[,i])),1-alpha)
  CVaRfr1_CA_bd[i]= wcvar(PGfr1_bd[,i],w=rep(1,length(PGfr1_bd[,i])),1-alpha)
  CVaRfr2_CA_bd[i]= wcvar(PGfr2_bd[,i],w=rep(1,length(PGfr2_bd[,i])),1-alpha)
  CVaRfr3_CA_bd[i]= wcvar(PGfr3_bd[,i],w=rep(1,length(PGfr3_bd[,i])),1-alpha)
  CVaRCont_CA_bd[i]= wcvar(PG_bd[,i],w=rep(1,length(PG_bd[,i])),1-alpha)
}

VaRCont_CA_bd
VaRfr1_CA_bd
VaRfr2_CA_bd
CVaRCont_CA_bd
CVaRfr1_CA_bd
CVaRfr2_CA_bd


#VaR Total

VaRTotal_CA_bd=wquantile(PGT_bd,w=rep(1,length(PGT_bd)),1-alpha)
CVaRTotal_CA_bd= wcvar(PGT_bd,w=rep(1,length(PGT_bd)),1-alpha)
VaRTotalfr1_CA_bd=wquantile(PGfr1T_bd,w=rep(1,length(PGfr1T_bd)),1-alpha)
CVaRTotalfr1_CA_bd= wcvar(PGfr1T_bd,w=rep(1,length(PGfr1T_bd)),1-alpha)
VaRTotalfr2_CA_bd=wquantile(PGfr2T_bd,w=rep(1,length(PGfr2T_bd)),1-alpha)
CVaRTotalfr2_CA_bd= wcvar(PGfr2T_bd,w=rep(1,length(PGfr2T_bd)),1-alpha)
VaRTotalfr3_CA_bd=wquantile(PGfr3T_bd,w=rep(1,length(PGfr3T_bd)),1-alpha)
CVaRTotalfr3_CA_bd= wcvar(PGfr3T_bd,w=rep(1,length(PGfr3T_bd)),1-alpha)

cbind(VaRTotal_CA_bd,sum(V0_bd), VaRCont_CA_bd, V0_bd)
cbind(CVaRTotal_CA_bd,sum(V0_bd), CVaRCont_CA_bd, V0_bd)
print("VaR con alisado")
print(cbind(VaRTotal_CA_bd,VaRTotalfr1_CA_bd,VaRTotalfr2_CA_bd,VaRTotalfr3_CA_bd))
print("CVaR con alisado")
print(cbind(CVaRTotal_CA_bd,CVaRTotalfr1_CA_bd,CVaRTotalfr2_CA_bd,CVaRTotalfr3_CA_bd))



[1] "VaR sin alisado"
     VaRTotal_bd VaRTotalfr1_bd VaRTotalfr2_bd VaRTotalfr3_bd
[1,]   -9265.609      -11135.56      -9813.498      -9813.498
[1] "CVaR sin alisado"
     CVaRTotal_bd CVaRTotalfr1_bd CVaRTotalfr2_bd CVaRTotalfr3_bd
[1,]    -9312.497       -12109.39       -10317.67       -10317.67
[1] "VaR con alisado"
     VaRTotal_CA_bd VaRTotalfr1_CA_bd VaRTotalfr2_CA_bd VaRTotalfr3_CA_bd
[1,]      -9278.587         -11135.56         -9948.996         -9948.996
[1] "CVaR con alisado"
     CVaRTotal_CA_bd CVaRTotalfr1_CA_bd CVaRTotalfr2_CA_bd CVaRTotalfr3_CA_bd
[1,]       -9318.149          -12109.39          -10379.12          -10379.12



Podemos observar como el bono cupón cero, bono M y bondes D, el más redituable y en menor tiempo es el bondes D, gracias a la cantidad generada y a su vez por el plazo en que paga cada bono (28 días).
Como podemos observar tenemos un VaR al 98% de confianza siendo este bajo condiciones de mercado normales de 9265.60 sin alisado y negativo, por lo que en este caso una ganancia, esto nos quiere decir que tenemos un 98% de probabilidad de que al siguiente periodo de tiempo esta gane eta cantidad anteriormente obtenida, ahora analizando el alisado, este es con un suavizamiento de los datos, siendo este porcentaje mayor al anterior previsto, con un 9278.58 de ganancia para nuestro portafolio de bonos para el siguiente periodo, siendo el total, y teniendo los resultados fraccionados, también con ganancias en todos los casos.
En el caso del CVaR, tenemos un valor un poco mayor, ya que este toma en cuenta la expectativa condicionada por el VaR, siendo aún más aproximado, al igual que en el caso anterior con el 98% de confianza, es decir que tiene un 98% de probabilidad de que en el siguiente periodo tenga una ganancia de 9312.49, que con el suavizamiento de los datos, es decir alisado, tendríamos un total para el portafolio de9318.14 de ganancias para el siguiente periodo, siendo el total, y teniendo los resultados fraccionados, también con ganancias en todos los casos.



##Swaps

In [0]:
%%R
#Cálculo de matriz de pérdidas y ganancias SWAP

#riesgo del swap
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[3]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)
Vfr2_sw=matrix(0,Ns,m)
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

DeltaX_s_sw=DeltaX_s[,sum(n_if[1:2],1):sum(n_if[1:3])]    #PASO CLAVE
x0_sw=as.numeric(c(XtfwdT[1,],Xvp[1,]))   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*(1+DeltaX_s_sw[i,])
  #PASO CLAVE
  V_sw[i,]=swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  #PASO CLAVE
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  PG_sw[i,]=V_sw[i,]-V0_sw
  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PGT_sw[i,]=sum(PG_sw[i,])
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
}



PG_sw[1:5,]
PGfr1_sw[1:5,]
PGfr2_sw[1:5,]
PGT_sw[1:5,]


#VaR por posición
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)
CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_sw[i]=equantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=equantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=equantile(PGfr2_sw[,i],1-alpha,Ns)
  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
}
VaRCont_sw
VaRfr1_sw
VaRfr2_sw
CVaRCont_sw
CVaRfr1_sw
CVaRfr2_sw


#VaR Total

VaRTotal_sw=equantile(PGT_sw,1-alpha,Ns)
CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
VaRTotalfr1_sw=equantile(PGfr1T_sw,1-alpha,Ns)
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
VaRTotalfr2_sw=equantile(PGfr2T_sw,1-alpha,Ns)
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])


cbind(VaRTotal_sw,sum(V0_sw), VaRCont_sw, V0_sw)
cbind(CVaRTotal_sw,sum(V0_sw), CVaRCont_sw, V0_sw)
print("VaR sin alisado")
print(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw))
print("CVaR sin alisado")
print(cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw))

###Con alisado

#VaR por posición
VaRCont_CA_sw=matrix(0,1,m)
VaRfr1_CA_sw=matrix(0,1,m)
VaRfr2_CA_sw=matrix(0,1,m)
CVaRCont_CA_sw=matrix(0,1,m)
CVaRfr1_CA_sw=matrix(0,1,m)
CVaRfr2_CA_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_CA_sw[i]=wquantile(PG_sw[,i],w=rep(1,length(PG_sw[,i])),1-alpha)
  VaRfr1_CA_sw[i]=wquantile(PGfr1_sw[,i],w=rep(1,length(PGfr1_sw[,i])),1-alpha)
  VaRfr2_CA_sw[i]=wquantile(PGfr2_sw[,i],w=rep(1,length(PGfr2_sw[,i])),1-alpha)
  CVaRfr1_CA_sw[i]= wcvar(PGfr1_sw[,i],w=rep(1,length(PGfr1_sw[,i])),1-alpha)
  CVaRfr2_CA_sw[i]= wcvar(PGfr2_sw[,i],w=rep(1,length(PGfr2_sw[,i])),1-alpha)
  CVaRCont_CA_sw[i]= wcvar(PG_sw[,i],w=rep(1,length(PG_sw[,i])),1-alpha)
}
VaRCont_CA_sw
VaRfr1_CA_sw
VaRfr2_CA_sw
CVaRCont_CA_sw
CVaRfr1_CA_sw
CVaRfr2_CA_sw


#VaR Total

VaRTotal_CA_sw=wquantile(PGT_sw,w=rep(1,length(PGT_sw)),1-alpha)
CVaRTotal_CA_sw= wcvar(PGT_sw,w=rep(1,length(PGT_sw)),1-alpha)
VaRTotalfr1_CA_sw=wquantile(PGfr1T_sw,w=rep(1,length(PGfr1T_sw)),1-alpha)
CVaRTotalfr1_CA_sw= wcvar(PGfr1T_sw,w=rep(1,length(PGfr1T_sw)),1-alpha)
VaRTotalfr2_CA_sw=wquantile(PGfr2T_sw,w=rep(1,length(PGfr2T_sw)),1-alpha)
CVaRTotalfr2_CA_sw= wcvar(PGfr2T_sw,w=rep(1,length(PGfr2T_sw)),1-alpha)


cbind(VaRTotal_CA_sw,sum(V0_sw), VaRCont_CA_sw, V0_sw)
cbind(CVaRTotal_CA_sw,sum(V0_sw), CVaRCont_CA_sw, V0_sw)
print("VaR sin alisado")
print(cbind(VaRTotal_CA_sw,VaRTotalfr1_CA_sw,VaRTotalfr2_CA_sw))
print("CVaR sin alisado")
print(cbind(CVaRTotal_CA_sw,CVaRTotalfr1_CA_sw,CVaRTotalfr2_CA_sw))


[1] "VaR sin alisado"
     VaRTotal_sw VaRTotalfr1_sw VaRTotalfr2_sw
[1,]   -2.570682      -2.570682    -0.01518482
[1] "CVaR sin alisado"
     CVaRTotal_sw CVaRTotalfr1_sw CVaRTotalfr2_sw
[1,]    -3.943025       -3.957843     -0.02965095
[1] "VaR sin alisado"
     VaRTotal_CA_sw VaRTotalfr1_CA_sw VaRTotalfr2_CA_sw
[1,]      -1.879324         -1.894364       -0.01697714
[1] "CVaR sin alisado"
     CVaRTotal_CA_sw CVaRTotalfr1_CA_sw CVaRTotalfr2_CA_sw
[1,]       -2.845433          -2.850967        -0.04052446


Tenemos un derivado con la característica de ser un Swap pagado con tasa fija
del 5.2% a un plazo de 588 días recibiendo entonces una tasa flotante, es decir,
estamos esperando que nuestra tasa flotante nos dé un mayor rendimiento del
que actualmente tenemos con la tasa fija, sin embargo, el análisis nos arroja que
tenemos un ValorSwap negativo de 28.3, es decir que la valoración de los
compromisos futuros de cobro es mayor que los compromisos futuros de pago
Además, en el cálculo de VaR y CVaR, tenemos valores negativos en cualquiera
de los casos, recordando que el VaR y el CVar nos indican los niveles de riesgo
que presentan los instrumentos financieros, en este caso al tener resultados
negativos podemos concluir que el interés que recibiremos al paso del periodo
establecido, será mejor que el que tenemos actualmente, pues un VaR negativo
implica que el portafolio, o en este caso, el instrumento tiene una alta probailidad
de obtener un beneficio.
Existe una mínima diferencia entre los datos obtenidos en el VaR y el CVaR, con
respecto al modelo alisado y sin alisado, si bien el modelo alisado permite reducir
al mínimo las fluctuaciones que se producen sobre los resultados obtenidos, pues
estas medidas pueden desviarse en mayor o menor medida de lo que se podría
considerar como lo esperado. En este caso, nuestro resultados de sin alisado
oscilan entre el 2 y 3 en un ambiente negativo, es decir, sin perdidas y cuando
aplicamos el alisado, sigue teniendo una tendencia negativa pero menor a lo que
se había obtenido, en conclusión, el instrumento swap presenta ganancias con
una mínima tendencia al riesgo.

##Opciones de tasas de interes

In [0]:
%%R
#Cálculo de matriz de pérdidas y ganancias Opciones Tasa de interés
#dimensión
m=ncol(plazos_oir)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[4]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Perdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s[,sum(n_if[1:3],1):sum(n_if[1:4])]  #PASO CLAVE
x0_oir=X_oir[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*(1+DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}



PG_oir[1:5,]
PGfr1_oir[1:5,]
PGfr2_oir[1:5,]
PGT_oir[1:5,]


#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}

VaRCont_oir
VaRfr1_oir
VaRfr2_oir
CVaRCont_oir
CVaRfr1_oir
CVaRfr2_oir


#VaR Total
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


cbind(VaRTotal_oir,sum(V0_oir), VaRCont_oir, V0_oir)
cbind(CVaRTotal_oir,sum(V0_oir), CVaRCont_oir, V0_oir)
print("VaR sin alisado")
print(cbind(VaRTotal_oir,VaRTotalfr1_oir,VaRTotalfr2_oir,VaRTotalfr3_oir))
print("CVaR sin alisado")
print(cbind(CVaRTotal_oir,CVaRTotalfr1_oir,CVaRTotalfr2_oir,CVaRTotalfr3_oir))

###Con alisado

#VaR por posición
VaRCont_CA_oir=matrix(0,1,m)
VaRfr1_CA_oir=matrix(0,1,m)
VaRfr2_CA_oir=matrix(0,1,m)
VaRfr3_CA_oir=matrix(0,1,m)
CVaRCont_CA_oir=matrix(0,1,m)
CVaRfr1_CA_oir=matrix(0,1,m)
CVaRfr2_CA_oir=matrix(0,1,m)
CVaRfr3_CA_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_CA_oir[i]=wquantile(PG_oir[,i],w=rep(1,length(PG_oir[,i])),1-alpha)
  VaRfr1_CA_oir[i]=wquantile(PGfr1_oir[,i],w=rep(1,length(PGfr1_oir[,i])),1-alpha)
  VaRfr2_CA_oir[i]=wquantile(PGfr2_oir[,i],w=rep(1,length(PGfr2_oir[,i])),1-alpha)
  VaRfr3_CA_oir[i]=wquantile(PGfr3_oir[,i],w=rep(1,length(PGfr3_oir[,i])),1-alpha)
  CVaRfr1_CA_oir[i]= wcvar(PGfr1_oir[,i],w=rep(1,length(PGfr1_oir[,i])),1-alpha)
  CVaRfr2_CA_oir[i]= wcvar(PGfr2_oir[,i],w=rep(1,length(PGfr2_oir[,i])),1-alpha)
  CVaRfr3_CA_oir[i]= wcvar(PGfr3_oir[,i],w=rep(1,length(PGfr3_oir[,i])),1-alpha)
  CVaRCont_CA_oir[i]= wcvar(PG_oir[,i],w=rep(1,length(PG_oir[,i])),1-alpha)
}

VaRCont_CA_oir
VaRfr1_CA_oir
VaRfr2_CA_oir
CVaRCont_CA_oir
CVaRfr1_CA_oir
CVaRfr2_CA_oir


#VaR Total
VaRTotal_CA_oir=wquantile(PGT_oir,w=rep(1,length(PGT_oir)),1-alpha)
CVaRTotal_CA_oir= wcvar(PGT_oir,w=rep(1,length(PGT_oir)),1-alpha)
VaRTotalfr1_CA_oir=wquantile(PGfr1T_oir,w=rep(1,length(PGfr1T_oir)),1-alpha)
CVaRTotalfr1_CA_oir= wcvar(PGfr1T_oir,w=rep(1,length(PGfr1T_oir)),1-alpha)
VaRTotalfr2_CA_oir=wquantile(PGfr2T_oir,w=rep(1,length(PGfr2T_oir)),1-alpha)
CVaRTotalfr2_CA_oir= wcvar(PGfr2T_oir,w=rep(1,length(PGfr2T_oir)),1-alpha)
VaRTotalfr3_CA_oir=wquantile(PGfr3T_oir,w=rep(1,length(PGfr3T_oir)),1-alpha)
CVaRTotalfr3_CA_oir= wcvar(PGfr3T_oir,w=rep(1,length(PGfr3T_oir)),1-alpha)


cbind(VaRTotal_CA_oir,sum(V0_oir), VaRCont_CA_oir, V0_oir)
cbind(CVaRTotal_CA_oir,sum(V0_oir), CVaRCont_CA_oir, V0_oir)
print("VaR con alisado")
print(cbind(VaRTotal_CA_oir,VaRTotalfr1_CA_oir,VaRTotalfr2_CA_oir,VaRTotalfr3_CA_oir))
print("CVaR con alisado")
print(cbind(CVaRTotal_CA_oir,CVaRTotalfr1_CA_oir,CVaRTotalfr2_CA_oir,CVaRTotalfr3_CA_oir))


[1] "VaR sin alisado"
     VaRTotal_oir VaRTotalfr1_oir VaRTotalfr2_oir VaRTotalfr3_oir
[1,]   -0.1094018    -0.007403825      -0.1094018     -0.02196278
[1] "CVaR sin alisado"
     CVaRTotal_oir CVaRTotalfr1_oir CVaRTotalfr2_oir CVaRTotalfr3_oir
[1,]    -0.1330923      -0.01132473        -0.133147     -0.001024698
[1] "VaR con alisado"
     VaRTotal_CA_oir VaRTotalfr1_CA_oir VaRTotalfr2_CA_oir VaRTotalfr3_CA_oir
[1,]      -0.1202319        -0.01169193         -0.1049403        -0.03727643
[1] "CVaR con alisado"
     CVaRTotal_CA_oir CVaRTotalfr1_CA_oir CVaRTotalfr2_CA_oir
[1,]       -0.1382646         -0.01331901          -0.1291895
     CVaRTotalfr3_CA_oir
[1,]         -0.07426871


Como podemos observar tenemos un VaR al 98% de confianza siendo este bajo condiciones de mercado normales de -10.94% sin alisado y negativo, por lo que en este caso una ganancia, esto nos quiere decir que tenemos un 98% de probabilidad de que al siguiente periodo de tiempo esta gane el porcentaje obtenido, ahora analizando el alisado, este es con un suavizamiento de los datos, siendo este porcentaje mayor al anterior previsto, con un -12.023% de ganancia para nuestro portafolio de opciones para el siguiente periodo, siendo el total, y teniendo los resultados fraccionados, también con ganancias en todos los casos.

En el caso del CVaR, tenemos un valor aún mayor, ya que este toma en cuenta la expectativa condicionada por el VaR, siendo aún más aproximado, al igual que en el caso anterior con el 98% de confianza, es decir que tiene un 98% de probabilidad de que en el siguiente periodo gane el porcentaje de ganancias del -13.30%, que con el suavizamiento de los datos, es decir alisado, tendríamos un total para el portafolio del -7.4% de ganancias para el siguiente periodo, siendo el total.